In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
# Load dataset
df = pd.read_csv('insurance.csv')

In [4]:
# -----------------------------
# 1. Dataset Summary Table
# -----------------------------
print("Dataset Info:")
print(df.info())
print("\nDataset Description:")
print(df.describe())

# Save dataset summary to CSV for table in LaTeX
df.describe().to_csv("dataset_summary.csv")

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
None

Dataset Description:
               age          bmi     children       charges
count  1338.000000  1338.000000  1338.000000   1338.000000
mean     39.207025    30.663397     1.094918  13270.422265
std      14.049960     6.098187     1.205493  12110.011237
min      18.000000    15.960000     0.000000   1121.873900
25%      27.000000    26.296250     0.000000   4740.287150
50%      39.000000    30.400000     1.000000   9382.033000
75%      51.000000    34.6

In [4]:
# -----------------------------
# 2. EDA Plots
# -----------------------------
plt.figure(figsize=(8,5))
sns.histplot(df['bmi'], bins=30, kde=True)
plt.title('Distribution of BMI')
plt.savefig("hist_bmi.png", dpi=300)
plt.close()

plt.figure(figsize=(8,5))
sns.histplot(df['charges'], bins=30, kde=True)
plt.title('Distribution of Charges')
plt.savefig("hist_charges.png", dpi=300)
plt.close()

plt.figure(figsize=(8,5))
sns.boxplot(x='smoker', y='charges', data=df)
plt.title('Charges by Smoking Status')
plt.savefig("box_smoker.png", dpi=300)
plt.close()

# Select only numerical columns for correlation heatmap
numerical_df = df.select_dtypes(include=np.number)

plt.figure(figsize=(8,6))
sns.heatmap(numerical_df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.savefig("correlation_heatmap.png", dpi=300)
plt.close()

In [5]:
# -----------------------------
# 3. Data Preprocessing
# -----------------------------
# One-hot encode categorical variables
df_encoded = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True)

# Convert boolean columns to integer (0s and 1s)
for col in ['sex_male', 'smoker_yes', 'region_northwest', 'region_southeast', 'region_southwest']:
    if col in df_encoded.columns:
        df_encoded[col] = df_encoded[col].astype(int)

X = df_encoded.drop('charges', axis=1)
y = df_encoded['charges']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# -----------------------------
# 4. Model Training & Evaluation
# -----------------------------
from sklearn.model_selection import cross_val_score

models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.01)
}

results = []
cv_results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results.append([name, r2, mae, rmse])

    # 5-fold cross-validation
    cv_score = cross_val_score(model, X, y, cv=5, scoring='r2').mean()
    cv_results.append([name, cv_score])

# Create and save results tables
results_df = pd.DataFrame(results, columns=['Model', 'R2', 'MAE', 'RMSE'])
results_df.to_csv("model_results.csv", index=False)

cv_df = pd.DataFrame(cv_results, columns=['Model', 'CV_R2'])
cv_df.to_csv("cv_results.csv", index=False)

In [7]:
# -----------------------------
# 5. Feature Importance - Random Forest
# -----------------------------
rf_model = models['Random Forest']
importances = rf_model.feature_importances_
feature_names = X.columns

plt.figure(figsize=(10,6))
sns.barplot(x=importances, y=feature_names)
plt.title('Feature Importance - Random Forest')
plt.tight_layout()
plt.savefig("feature_importance.png", dpi=300)
plt.close()

In [8]:
# -----------------------------
# 6. SHAP Feature Importance
# -----------------------------
import shap
import pandas as pd

explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test)

plt.figure(figsize=(10,6))
shap.summary_plot(shap_values, X_test, show=False)
plt.savefig("shap_summary.png", dpi=300, bbox_inches='tight')
plt.close()

In [27]:
# -----------------------------
# 7. Predicted vs Actual Charges
# -----------------------------
y_pred_rf = rf_model.predict(X_test)

plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred_rf, alpha=0.6)
plt.xlabel('Actual Charges')
plt.ylabel('Predicted Charges')
plt.title('Predicted vs Actual Charges (Random Forest)')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.tight_layout()
plt.savefig("predicted_vs_actual.png", dpi=300)
plt.close()

In [28]:
# -----------------------------
# 8. Residual Plot
# -----------------------------
residuals = y_test - y_pred_rf
plt.figure(figsize=(8,6))
sns.scatterplot(x=y_pred_rf, y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Charges')
plt.ylabel('Residuals')
plt.title('Residual Plot (Random Forest)')
plt.tight_layout()
plt.savefig("residual_plot.png", dpi=300)
plt.close()

In [29]:
# -----------------------------
# 9. Lasso & Ridge Coefficient Plots
# -----------------------------
ridge_model = models['Ridge Regression']
lasso_model = models['Lasso Regression']

plt.figure(figsize=(10,6))
sns.barplot(x=ridge_model.coef_, y=X.columns)
plt.title('Ridge Regression Coefficients')
plt.tight_layout()
plt.savefig("ridge_coefficients.png", dpi=300)
plt.close()

plt.figure(figsize=(10,6))
sns.barplot(x=lasso_model.coef_, y=X.columns)
plt.title('Lasso Regression Coefficients')
plt.tight_layout()
plt.savefig("lasso_coefficients.png", dpi=300)
plt.close()